<a href="https://colab.research.google.com/github/sangchoi93/data_minor/blob/Dtree/DecisionTreeProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold, cross_val_score
from sklearn.model_selection import GridSearchCV
import math
import time

In [33]:
df=pd.read_csv('heart_failure_clinical_records_dataset.csv')
df['DEATH_EVENT'].value_counts()

0    203
1     96
Name: DEATH_EVENT, dtype: int64

In [34]:
df.describe()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
count,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.00000,299.000000,299.000000,299.00000,299.000000,299.00000
mean,60.833893,0.431438,581.839465,0.418060,38.083612,0.351171,263358.029264,1.39388,136.625418,0.648829,0.32107,130.260870,0.32107
std,11.894809,0.496107,970.287881,0.494067,11.834841,0.478136,97804.236869,1.03451,4.412477,0.478136,0.46767,77.614208,0.46767
min,40.000000,0.000000,23.000000,0.000000,14.000000,0.000000,25100.000000,0.50000,113.000000,0.000000,0.00000,4.000000,0.00000
25%,51.000000,0.000000,116.500000,0.000000,30.000000,0.000000,212500.000000,0.90000,134.000000,0.000000,0.00000,73.000000,0.00000
50%,60.000000,0.000000,250.000000,0.000000,38.000000,0.000000,262000.000000,1.10000,137.000000,1.000000,0.00000,115.000000,0.00000
75%,70.000000,1.000000,582.000000,1.000000,45.000000,1.000000,303500.000000,1.40000,140.000000,1.000000,1.00000,203.000000,1.00000
max,95.000000,1.000000,7861.000000,1.000000,80.000000,1.000000,850000.000000,9.40000,148.000000,1.000000,1.00000,285.000000,1.00000


In [35]:
X = df[['age', 'anaemia', 'creatinine_phosphokinase', 'diabetes',
       'ejection_fraction', 'high_blood_pressure', 'platelets',
       'serum_creatinine', 'serum_sodium', 'sex', 'smoking', 'time']].values


In [36]:
y = df['DEATH_EVENT'].values


**Our own implementation of Decision Tree algorithm**

In [37]:
def check_purity(data):
    label_column = data[:,-1]
    unique_classes = np.unique(label_column)

    if len(unique_classes) == 1:
        return True
    else:
        return False

def classify_data(data):
    label_column = data[:,-1]
    unique_classes, counts_unique_classes = np.unique(label_column, return_counts=True)

    index = counts_unique_classes.argmax()
    classification = unique_classes[index]
    return classification

def get_potential_splits(data):
    potential_splits = {}
    _, n_columns = data.shape
    
    for column_index in range(n_columns - 1):
        potential_splits[column_index] = []
        values = data[:, column_index]
        unique_values = np.unique(values)
        #potential_splits[column_index] = unique_values
        for index in range(len(unique_values)):
            if index != 0:
                current_value = unique_values[index]
                previous_value = unique_values[index - 1]
                potential_split = (current_value + previous_value) / 2
                
                potential_splits[column_index].append(potential_split)        
    return potential_splits

def split_data(data, split_column, split_value):
    
    split_column_values = data[:,split_column]
    
    data_below = data[split_column_values <= split_value]
    data_above = data[split_column_values > split_value]

    return data_below, data_above

def calculate_entropy(data):
    label_column = data[:,-1]
    _, counts = np.unique(label_column, return_counts=True)

    probabilities = counts / counts.sum()
    entropy = sum(probabilities * -np.log2(probabilities))
    return entropy

def calculate_overall_entropy(data_below, data_above):
    n_data_points = len(data_below) + len(data_above)
    p_data_below = len(data_below) / n_data_points
    p_data_above = len(data_above) / n_data_points

    overall_entropy = (p_data_below * calculate_entropy(data_below)) + (p_data_above * calculate_entropy(data_above))

    return overall_entropy

def determine_best_split(data, potential_splits):
    overall_entropy = 9999
    for column_index in potential_splits:
        for value in potential_splits[column_index]:
            data_below, data_above = split_data(data, split_column=column_index, split_value=value)
            current_overall_entropy = calculate_overall_entropy(data_below,data_above)

            if current_overall_entropy <= overall_entropy:
                overall_entropy = current_overall_entropy
                best_split_column = column_index
                best_split_value = value
    return best_split_column, best_split_value

def decision_tree_algorithm(df, counter=0, min_samples = 2, max_depth=5):

    if counter == 0:
        global COLUMN_HEADERS
        COLUMN_HEADERS = df.columns
        data = df.values
    else:
        data = df

    if (check_purity(data)) or (len(data) < min_samples) or (counter == max_depth):
        classification = classify_data(data)
        return classification
    else:
        counter +=1
        
        potential_splits = get_potential_splits(data)
        split_column, split_value = determine_best_split(data, potential_splits)
        data_below, data_above = split_data(data, split_column, split_value)

        feature_name = COLUMN_HEADERS[split_column]
        question = '{} <= {}'.format(feature_name, split_value)
        sub_tree = {question:[]}

        yes_answer = decision_tree_algorithm(data_below, counter,min_samples,max_depth)
        no_answer = decision_tree_algorithm(data_above, counter,min_samples,max_depth)

        if yes_answer == no_answer:
            sub_tree = yes_answer
        else:
            sub_tree[question].append(yes_answer)
            sub_tree[question].append(no_answer)

        return sub_tree        

def predict_example(example, tree):
    
    # tree is just a root node
    if not isinstance(tree, dict):
        return tree
    
    question = list(tree.keys())[0]
    feature_name, comparison_operator, value = question.split(" ")

    # ask question
    if comparison_operator == "<=":
        if example[feature_name] <= float(value):
            answer = tree[question][0]
        else:
            answer = tree[question][1]
    
    # feature is categorical
    else:
        if str(example[feature_name]) == value:
            answer = tree[question][0]
        else:
            answer = tree[question][1]

    # base case
    if not isinstance(answer, dict):
        return answer
    
    # recursive part
    else:
        residual_tree = answer
        return predict_example(example, residual_tree)

def make_predictions(df, tree):
    
    if len(df) != 0:
        predictions = df.apply(predict_example, args=(tree,), axis=1)
    else:
        # "df.apply()"" with empty dataframe returns an empty dataframe,
        # but "predictions" should be a series instead
        predictions = pd.Series()
        
    return predictions

def calculate_accuracy(df, tree):
    predictions = make_predictions(df, tree)
    predictions_correct = predictions == df.DEATH_EVENT
    accuracy = predictions_correct.mean()
    
    return accuracy

**Splitting data for training and testing with kfold**

In [38]:
kfold = KFold(n_splits=10)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)
print('Train set:', X_train.shape, y_train.shape)
print('Test set:', X_test.shape, y_test.shape)

Train set: (224, 12) (224,)
Test set: (75, 12) (75,)


**Running both algorithms with kfold splits**

In [39]:
kfold.split(X_train)
dtc_accuracy_model = []
dtc_model_time = []
mydtc_accuracy_model = []
mydtc_model_time = []
for train_index, test_index in kfold.split(X):
    
    # Split train-test
    X_train, X_test = df.iloc[train_index], df.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    start = int(round(time.time() * 1000))
    dtc = DecisionTreeClassifier(criterion='entropy',random_state=0,max_depth=5)
    # Train the model
    tree_model = dtc.fit(X_train, y_train)
    
    # Append to accuracy_model the accuracy of the model
    dtc_accuracy_model.append(accuracy_score(y_test, tree_model.predict(X_test), normalize=True)*100)
    time_dtc = int(round(time.time() * 1000)) - start
    dtc_model_time.append(time_dtc)

    X_train.columns = df.columns
    start = int(round(time.time() * 1000))
    tree = decision_tree_algorithm(X_train)
    mydtc_accuracy_model.append(calculate_accuracy(X_test,tree)*100)
    time_mydtc = int(round(time.time() * 1000)) - start
    mydtc_model_time.append(time_mydtc)

    diff = [(i / j)*100 for i, j in zip(mydtc_model_time,dtc_model_time)] 
results_df = pd.DataFrame()
results_df['dtc_accuracy_model'] = dtc_accuracy_model
results_df['dtc_model_time'] = dtc_model_time
results_df['mydtc_accuracy_model'] = mydtc_accuracy_model
results_df['mydtc_model_time'] = mydtc_model_time
results_df['%diff'] = diff

results_df


,dtc_accuracy_model,dtc_model_time,mydtc_accuracy_model,mydtc_model_time,%diff
0,100.0,4,80.000000,346,8650.000000
1,100.0,3,80.000000,320,10666.666667
2,100.0,2,83.333333,335,16750.000000
3,100.0,2,73.333333,342,17100.000000
4,100.0,3,83.333333,312,10400.000000
5,100.0,3,80.000000,318,10600.000000
6,100.0,2,73.333333,325,16250.000000
7,100.0,3,76.666667,316,10533.333333
8,100.0,3,80.000000,304,10133.333333
9,100.0,2,93.103448,354,17700.000000


**Tree reporesentation of our algorithm**

In [40]:
tree

{'time <= 67.5': [{'platelets <= 214500.0': [1.0,
    {'platelets <= 224500.0': [{'time <= 57.0': [0.0, 1.0]},
      {'age <= 66.5': [{'creatinine_phosphokinase <= 85.5': [0.0, 1.0]},
        1.0]}]}]},
  {'serum_creatinine <= 1.55': [{'ejection_fraction <= 27.5': [{'time <= 78.5': [1.0,
        0.0]},
      {'age <= 79.0': [0.0, {'platelets <= 239000.0': [0.0, 1.0]}]}]},
    {'ejection_fraction <= 22.5': [1.0,
      {'creatinine_phosphokinase <= 72.5': [0.0,
        {'high_blood_pressure <= 0.5': [0.0, 1.0]}]}]}]}]}